<a href="https://colab.research.google.com/github/Dagmara-Czarnecka/non-governmental-organizations/blob/main/non_governmental_organizations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup

URL = 'https://spis.ngo.pl/?cat[2384]=2753'

response = requests.get(URL)
category = BeautifulSoup(response.text, 'html.parser')

list_of_organizations = category.select_one('div.relative.mb5')

print('Znaleziono', len(list_of_organizations), 'organizacji')


Znaleziono 7 organizacji


In [2]:
organizations = list_of_organizations.select('li.pv3.bb')


In [3]:
urls = []

for organization in organizations:
    urls.append(
        organization.select_one('a')['href']
    )


In [ ]:
from selenium.webdriver import Firefox  #you can also use a different browser
from selenium.webdriver.firefox.service import Service
from selenium.common.exceptions import NoSuchElementException

def get_organization_data(url):
    data = {
        'Adres email': '',
        'Adresy www': [],
        'Telefony': [],
        'KRS': '',
        'REGON': '',
        'NIP': '',
        'Rok powstania': '',
    }

    service = Service('./geckodriver') # correct the path if necessary
    browser = Firefox(service=service)  
    browser.get(url)

    table = browser.find_element('css selector', 'div.ba.b--light-gray.pa3.pt0')
    rows = table.find_elements('css selector','div.f7-f6-xl')


    for row in rows:
        try:
            label = row.find_element('css selector', '.pr2')
            value = row.find_element('css selector', '.tr.grow-1')
        except NoSuchElementException:

            continue

 
        if label.text == 'E-mail':
            value.find_element('tag name','span').click()
            data['Adres email'] = value.text

   
        elif label.text == 'Telefon':
            data['Telefony'].append(value.text)

        elif label.text == 'WWW':
            data['Adresy www'].append(value.text)

        elif label.text == 'KRS':
            data['KRS'] = value.text

        elif label.text == 'REGON':
            data['REGON'] = value.text

        elif label.text == 'NIP':
            data['NIP'] = value.text

        elif label.text == 'Rok powstania':
            data['Rok powstania'] = value.text

    
    browser.quit()
    return data


In [ ]:
import csv

with open('report.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    for organization_url in urls:
        data = get_organization_data(organization_url)
        writer.writerow([
            data['Adres email'],
            ' '.join(data['Adresy www']),
            ', '.join(data['Telefony']),
            data['KRS'],
            data['REGON'],
            data['NIP'],
            data['Rok powstania'],
        ])
